In [4]:
from pyspark.sql import SparkSession
from delta import *

builder = SparkSession.builder.appName("Delta Lake Lab") \
.config("spark.sql.extensions",
"io.delta.sql.DeltaSparkSessionExtension") \
.config("spark.sql.catalog.spark_catalog",
"org.apache.spark.sql.delta.catalog.DeltaCatalog")
spark = configure_spark_with_delta_pip(builder).getOrCreate()


In [5]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema = StructType([
    StructField("id", IntegerType(), nullable=False),
    StructField("name", StringType(), nullable=False),
    StructField("location", StringType(), nullable=True),
    StructField("build_year", IntegerType(), nullable=False)
])

data = [
    (1, "Софійський собор", "Київ", 1037),
    (2, "Львівський оперний театр", "Львів", 1900),
    (3, "Кам'янець-Подільська фортеця", "Кам'янець-Подільський", 1362)
]

df = spark.createDataFrame(data, schema=schema)
df.show()


+---+--------------------+--------------------+----------+
| id|                name|            location|build_year|
+---+--------------------+--------------------+----------+
|  1|    Софійський собор|                Київ|      1037|
|  2|Львівський оперни...|               Львів|      1900|
|  3|Кам'янець-Подільс...|Кам'янець-Подільс...|      1362|
+---+--------------------+--------------------+----------+



In [6]:
df.write.format("delta").mode("overwrite").save("/tmp/delta-cultural-heritage1")


In [7]:
delta_table = spark.read.format("delta").load("/tmp/delta-cultural-heritage1")
delta_table.show()
delta_table.printSchema()


+---+--------------------+--------------------+----------+
| id|                name|            location|build_year|
+---+--------------------+--------------------+----------+
|  3|Кам'янець-Подільс...|Кам'янець-Подільс...|      1362|
|  2|Львівський оперни...|               Львів|      1900|
|  1|    Софійський собор|                Київ|      1037|
+---+--------------------+--------------------+----------+

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- build_year: integer (nullable = true)



In [8]:
new_data = [
    (4, "Херсонеський музей-заповідник", "Севастополь", 1892)
]
new_df = spark.createDataFrame(new_data, schema=schema)
new_df.write.format("delta").mode("append").save("/tmp/delta-cultural-heritage1")


In [9]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, "/tmp/delta-cultural-heritage1")

deltaTable.update(
    condition="id == 2",
    set={"build_year": "1899"}
)


In [10]:
updated_table = spark.read.format("delta").load("/tmp/delta-cultural-heritage1")
updated_table.show()

+---+--------------------+--------------------+----------+
| id|                name|            location|build_year|
+---+--------------------+--------------------+----------+
|  3|Кам'янець-Подільс...|Кам'янець-Подільс...|      1362|
|  4|Херсонеський музе...|         Севастополь|      1892|
|  2|Львівський оперни...|               Львів|      1899|
|  1|    Софійський собор|                Київ|      1037|
+---+--------------------+--------------------+----------+



In [13]:
incorrect_data = [
    (5, "Замок Паланок", "Мукачево", None)
]
incorrect_df = spark.createDataFrame(incorrect_data, schema=schema)

try:
    incorrect_df.write.format("delta").mode("append").save("/tmp/delta-cultural-heritage1")
except Exception as e:
    print("ERROR OCCURED", e)


PySparkValueError: [CANNOT_BE_NONE] Argument `obj` can not be None.

In [14]:
deltaTable.history().show()


+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      2|2024-10-23 06:11:...|  NULL|    NULL|   UPDATE|{predicate -> ["(...|NULL|    NULL|     NULL|          1|  Serializable|        false|{numRemovedFiles ...|        NULL|Apache-Spark/3.5....|
|      1|2024-10-23 06:10:...|  NULL|    NULL|    WRITE|{mode -> Append, ...|NULL|    NULL|     NULL|          0|  Serializable|         true|{numFiles -> 2, n...|        NULL|Apache-Spark/3.5....|
|      0|2

In [15]:
previous_version = spark.read.format("delta").option("versionAsOf", 0).load("/tmp/delta-cultural-heritage1")
previous_version.show()


+---+--------------------+--------------------+----------+
| id|                name|            location|build_year|
+---+--------------------+--------------------+----------+
|  3|Кам'янець-Подільс...|Кам'янець-Подільс...|      1362|
|  2|Львівський оперни...|               Львів|      1900|
|  1|    Софійський собор|                Київ|      1037|
+---+--------------------+--------------------+----------+



In [16]:
spark.stop()